In [92]:
import torch
import numpy as np
import os
from torch import nn
from dataset import CIFAR10_custom
from transforms import CompressedToTensor, ZigZagOrder, ChooseAC, FlattenZigZag
from torchvision.transforms import Compose


torch.set_printoptions(threshold=10_000)
np.set_printoptions(threshold=np.inf)

In [93]:
download_path = os.path.join('..', 'data', 'cifar10')

if not os.path.exists(download_path):
    os.makedirs(download_path)

In [ ]:
# transform = Compose([CompressedToTensor(),
#                      ZigZagOrder(),
#                      ChooseAC(5),
#                      FlattenZigZag()])


transform = Compose([CompressedToTensor(),
                     ZigZagOrder(),
                     ChooseAC(5)])


cifar = CIFAR10_custom(root=download_path, train=True, transform=transform, target_transform=None, download = False, compression=None)

In [ ]:
cifar.to_ycbcr(in_place=True)
cifar.compress(in_place=True)

torch.Size([2, 3, 16, 6])
torch.Size([2, 48, 6])


torch.Size([2, 48, 248])

tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]],

         [[ 9., 10.],
          [11., 12.]],

         [[13., 14.],
          [15., 16.]]],


        [[[17., 18.],
          [19., 20.]],

         [[21., 22.],
          [23., 24.]],

         [[25., 26.],
          [27., 28.]],

         [[29., 30.],
          [31., 32.]]],


        [[[33., 34.],
          [35., 36.]],

         [[37., 38.],
          [39., 40.]],

         [[41., 42.],
          [43., 44.]],

         [[45., 46.],
          [47., 48.]]]])

tensor([[ 1.,  2.,  3.,  4., 17., 18., 19., 20., 33., 34., 35., 36.],
        [ 5.,  6.,  7.,  8., 21., 22., 23., 24., 37., 38., 39., 40.],
        [ 9., 10., 11., 12., 25., 26., 27., 28., 41., 42., 43., 44.],
        [13., 14., 15., 16., 29., 30., 31., 32., 45., 46., 47., 48.]])
torch.Size([4, 12])
